In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sys 
import argparse
import logging
import os
import pickle
import random
import torch
import json
import numpy as np
import pandas as pd
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,BertConfig,BertModel, AlbertConfig, AlbertModel,DebertaV2Config,DebertaV2Model,
                              RobertaConfig, RobertaModel, RobertaTokenizer)
from sklearn.model_selection import GroupKFold,StratifiedGroupKFold
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import math
from collections import defaultdict
from itertools import chain
from torch.optim import Optimizer
import torch
import warnings

2023-05-03 11:10:41.870806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:10:41.995600: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 11:10:42.683461: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zhouyuanzhe/hxdoc:
2023-05-03 11:10:42.683554: W tensorflow/compiler/xla/stream_e

In [3]:
label_smoothing = 0.75
num_class  = 250
num_landmark = 543
max_length = 256
# point_dim  = 1323
embed_dim  = 240
num_head   = 16
num_block  = 3
root_dir = '.'

LHAND = np.arange(468, 489).tolist() # 21
RHAND = np.arange(522, 543).tolist() # 21
POSE  = np.arange(489, 522).tolist() # 33
FACE  = np.arange(0,468).tolist()    #468

REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
][::2]
LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
][::2]
NOSE=[
    1,2,98,327
]
SLIP = [
    78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    191, 80, 81, 82, 13, 312, 311, 310, 415,
]
SPOSE = (np.array([
    11,13,15,12,14,16,23,24,
])+489).tolist()

BODY = REYE+LEYE+NOSE+SLIP+SPOSE

def get_indexs(L):
    return sorted([i + j * len(L) for i in range(len(L)) for j in range(len(L)) if i>j])

DIST_INDEX = get_indexs(RHAND)

LIP_DIST_INDEX = get_indexs(SLIP)

POSE_DIST_INDEX = get_indexs(SPOSE)

EYE_DIST_INDEX = get_indexs(REYE)

NOSE_DIST_INDEX = get_indexs(NOSE)

HAND_START = [0,1,2,3,5,6,7,9,10,11,13,14,15,17,18,19,0,5,9,13,0]
HAND_END = [1,2,3,4,6,7,8,10,11,12,14,15,16,18,19,20,5,9,13,17,17]

point_dim = point_dim2 = len(LHAND+RHAND+REYE+LEYE+NOSE+SLIP+SPOSE)*2+len(LHAND+RHAND)*2+len(RHAND)+len(DIST_INDEX)*2+len(EYE_DIST_INDEX)*2+len(LIP_DIST_INDEX)+len(POSE_DIST_INDEX)

def pack_seq(
	seq,
):
	length = [min(len(s), max_length)  for s in seq]
	batch_size = len(seq)
	K = seq[0].shape[1]
	L = max(length)
	#print(length)

	x = torch.zeros((batch_size, L, point_dim))#.to(seq[0].device)
	x_mask = torch.zeros((batch_size, L))#.to(seq[0].device)
	for b in range(batch_size):
		l = length[b]
		x[b, :l] = seq[b][:l,:]
		x_mask[b, l:] = 1
	x_mask = (x_mask>0.5)

	return x, x_mask

def positional_encoding(length, embed_dim):
	dim = embed_dim//2
	position = np.arange(length)[:, np.newaxis]     # (seq, 1)
	dim = np.arange(dim)[np.newaxis, :]/dim   # (1, dim)
	angle = 1 / (10000**dim)         # (1, dim)
	angle = position * angle    # (pos, dim)
	pos_embed = np.concatenate(
		[np.sin(angle), np.cos(angle)],
		axis=-1
	)
	pos_embed = torch.from_numpy(pos_embed).float()
	return pos_embed

class FCLayer(nn.Module):
    def __init__(self, embed_dim, hidden_dim, split=4):
        super().__init__()
        self.dim = embed_dim//4
        self.mlp = nn.ModuleList(
            [nn.Linear(embed_dim//4, hidden_dim//4) for i in range(split)]
        )
        self.split=split
    def forward(self, x):
        output = []
        for i in range(self.split):
            output.append(self.mlp[i](x[:,:,self.dim*i:self.dim*i+self.dim]))
        x = torch.cat(output, -1)
        return x

class TransformerBlock(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=embed_dim // num_head,
			v_dim=embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x, x_mask=None):
		x = x + self.attn((self.norm1(x)), x_mask)
		x = x + self.ffn((self.norm2(x)))
		return x
    
class MyMultiHeadAttention(nn.Module):
	def __init__(self,
			embed_dim,
			out_dim,
			qk_dim,
			v_dim,
			num_head,
		):
		super().__init__()
		self.embed_dim = embed_dim
		self.num_head  = num_head
		self.qk_dim = qk_dim
		self.v_dim  = v_dim

		self.q = nn.Linear(embed_dim, qk_dim*num_head)
		self.k = nn.Linear(embed_dim, qk_dim*num_head)
		self.v = nn.Linear(embed_dim, v_dim*num_head)

		self.out = nn.Linear(v_dim*num_head, out_dim)
		self.scale = 1/(qk_dim**0.5)

	#https://github.com/pytorch/pytorch/issues/40497
	def forward(self, x, x_mask):
		B,L,dim = x.shape
		#out, _ = self.mha(x,x,x, key_padding_mask=x_mask)
		num_head = self.num_head
		qk_dim = self.qk_dim
		v_dim = self.v_dim

		q = self.q(x)
		k = self.k(x)
		v = self.v(x)
		q = q.reshape(B, L, num_head, qk_dim).permute(0,2,1,3).contiguous()
		k = k.reshape(B, L, num_head, qk_dim).permute(0,2,3,1).contiguous()
		v = v.reshape(B, L, num_head, v_dim ).permute(0,2,1,3).contiguous()

		dot = torch.matmul(q, k) *self.scale  # H L L
		x_mask = x_mask.reshape(B,1,1,L).expand(-1,num_head,L,-1)
		#dot[x_mask]= -1e4
		dot.masked_fill_(x_mask, -1e4)
		attn = F.softmax(dot, -1)    # L L

		v = torch.matmul(attn, v)  # L H dim
		v = v.permute(0,2,1,3).reshape(B,L, v_dim*num_head).contiguous()
		out = self.out(v)

		return out
    

class FeedForward(nn.Module):
	def __init__(self, embed_dim, hidden_dim):
		super().__init__()
		self.mlp = nn.Sequential(
			nn.Linear(embed_dim, hidden_dim),
            # FCLayer(embed_dim, hidden_dim,4),
			nn.ReLU(inplace=True),
            # FCLayer(embed_dim, hidden_dim,4),
			nn.Linear(hidden_dim, embed_dim),
		)
	def forward(self, x):
		return self.mlp(x)

    

In [4]:
class InputNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.max_length = max_length 
  
    def forward(self, xyz):
        xyz = xyz - xyz[~torch.isnan(xyz)].mean(0,keepdim=True) #noramlisation to common maen
        xyz = xyz / xyz[~torch.isnan(xyz)].std(0, keepdim=True)

        LIP = [
            78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
            191, 80, 81, 82, 13, 312, 311, 310, 415,
        ]
        #LHAND = np.arange(468, 489).tolist()
        #RHAND = np.arange(522, 543).tolist()

        lip = xyz[:, LIP]
        lhand = xyz[:, 468:489]
        rhand = xyz[:, 522:543]
        xyz = torch.cat([  # (none, 82, 3)
            lip,
            lhand,
            rhand,
        ], 1)
        xyz[torch.isnan(xyz)] = 0
        x = xyz[:self.max_length]
        return x


In [5]:


#overwrite the model used in training ....
class MyMultiHeadAttention(nn.Module):
	def __init__(self,
			embed_dim,
			out_dim,
			qk_dim,
			v_dim,
			num_head,
		):
		super().__init__()
		self.embed_dim = embed_dim
		self.num_head  = num_head
		self.qk_dim = qk_dim
		self.v_dim  = v_dim

		self.q = nn.Linear(embed_dim, qk_dim*num_head)
		self.k = nn.Linear(embed_dim, qk_dim*num_head)
		self.v = nn.Linear(embed_dim, v_dim*num_head)

		self.out = nn.Linear(v_dim*num_head, out_dim)
		self.scale = 1/(qk_dim**0.5)

	#https://github.com/pytorch/pytorch/issues/40497
	def forward(self, x):
		B,L,dim = x.shape
		#out, _ = self.mha(x,x,x, key_padding_mask=x_mask)
		num_head = self.num_head
		qk_dim = self.qk_dim
		v_dim = self.v_dim

		q = self.q(x)
		k = self.k(x)
		v = self.v(x)
		q = q.reshape(B, L, num_head, qk_dim).permute(0,2,1,3).contiguous()
		k = k.reshape(B, L, num_head, qk_dim).permute(0,2,3,1).contiguous()
		v = v.reshape(B, L, num_head, v_dim ).permute(0,2,1,3).contiguous()

		dot = torch.matmul(q, k) *self.scale  # H L L
		#x_mask = x_mask.reshape(B,1,1,L).expand(-1,num_head,L,-1)
		#dot[x_mask]= -1e4
		#dot.masked_fill_(x_mask, -1e4)
		attn = F.softmax(dot, -1)    # L L

		v = torch.matmul(attn, v)  # L H dim
		v = v.permute(0,2,1,3).reshape(B,L, v_dim*num_head).contiguous()
		out = self.out(v)

		return out
    

# remove mask
class TransformerBlock2(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=embed_dim // num_head,
			v_dim=embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x):
		x = x + self.attn((self.norm1(x)))
		x = x + self.ffn((self.norm2(x)))
		return x

class TransformerBlock3(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=24, #embed_dim // num_head,
			v_dim=24, #embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x):
		x = x + self.attn((self.norm1(x)))
		x = x + self.ffn((self.norm2(x)))
		return x

class TransformerBlock(nn.Module):
	def __init__(self,
	    embed_dim,
        num_head,
        out_dim,
	):
		super().__init__()
		self.attn  = MyMultiHeadAttention(
			embed_dim=embed_dim,
			out_dim=embed_dim,
			qk_dim=embed_dim // num_head,
			v_dim=embed_dim // num_head,
			num_head=num_head,

		)
		self.ffn   = FeedForward(embed_dim, out_dim*2)
		self.norm1 = nn.LayerNorm(embed_dim)
		self.norm2 = nn.LayerNorm(out_dim)

	def forward(self, x):
		x = x + self.attn((self.norm1(x)))
		x = x + self.ffn((self.norm2(x)))
		return x

class XEmbed(nn.Module):
    def __init__(self,):
        super().__init__()
        self.v = nn.Sequential(
 			nn.Linear(point_dim, embed_dim*2, bias=True),
 			nn.LayerNorm(embed_dim*2),
 			nn.ReLU(inplace=True),
 			nn.Linear(embed_dim*2, embed_dim, bias=True),
 			nn.LayerNorm(embed_dim),
 			nn.ReLU(inplace=True),
 		)
    def forward(self, x):
        v = self.v(x)
        x = v
        return x


class XEmbed2(nn.Module):
    def __init__(self,):
        super().__init__()
        self.v = nn.Sequential(
 			nn.Linear(point_dim, 480*2, bias=True),
 			nn.LayerNorm(480*2),
 			nn.ReLU(inplace=True),
 			nn.Linear(480*2, 480, bias=True),
 			nn.LayerNorm(480),
 			nn.ReLU(inplace=True),
 		)
    def forward(self, x):
        v = self.v(x)
        x = v
        return x

class SingleNet(nn.Module):

    def __init__(self, num_class=num_class):
        super().__init__()
        self.num_block = 3
        self.embed_dim = embed_dim
        self.num_head  = num_head
        self.max_length = max_length
        self.num_point = point_dim

        pos_embed = positional_encoding(max_length, self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed = XEmbed()

        self.encoder = nn.ModuleList([
            TransformerBlock(
                self.embed_dim,
                self.num_head,
                self.embed_dim
            ) for i in range(self.num_block)
        ])
        self.logit = nn.Linear(self.embed_dim, num_class)

    def forward(self, xyz):
        L,D = xyz.shape
        xyz = xyz.reshape(1,L,D)
        
        x = self.x_embed(xyz)
        
        x = x + self.pos_embed[:L].reshape(1,L,embed_dim)
        x = torch.cat([
                self.cls_embed.reshape(1,1,embed_dim),
                x
            ],1)

        #for block in self.encoder: x = block(x) #remove tflite loop
        x = self.encoder[0](x)
        x = self.encoder[1](x)
        x = self.encoder[2](x)
        
        logit = self.logit(x.mean(1))
        return logit
    

def positional_encoding1(length, embed_dim):
	dim = embed_dim//2
	position = np.arange(length)[:, np.newaxis]     # (seq, 1)
	dim = np.arange(dim)[np.newaxis, :]/dim   # (1, dim)
	angle = 1 / (500**dim)         # (1, dim)
	angle = position * angle    # (pos, dim)
	pos_embed = np.concatenate(
		[np.sin(angle), np.cos(angle)],
		axis=-1
	)
	pos_embed = torch.from_numpy(pos_embed).float()
	return pos_embed

class SingleNet2(nn.Module):

    def __init__(self, max_length):
        super().__init__()
        self.num_block = 3
        self.embed_dim = 480
        self.num_head  = 16
        self.max_length = max_length
        self.num_point = point_dim

        pos_embed = positional_encoding(max_length, self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed = XEmbed2()

        self.encoder = nn.ModuleList([
            TransformerBlock2(
                self.embed_dim,
                self.num_head,
                self.embed_dim
            ) for i in range(self.num_block)
        ])
        self.logit = nn.Linear(self.embed_dim, num_class)

    def forward(self, xyz):
        L,D = xyz.shape
        xyz = xyz.reshape(1,L,D)
        
        x = self.x_embed(xyz)
        
        x = x + self.pos_embed[:L].reshape(1,L,self.embed_dim)
        x = torch.cat([
                self.cls_embed.reshape(1,1,self.embed_dim),
                x
            ],1)

        #for block in self.encoder: x = block(x) #remove tflite loop
        x = self.encoder[0](x)
        x = self.encoder[1](x)
        x = self.encoder[2](x)

        logit = self.logit(x.mean(1))
        return logit
    
class SingleNet3(nn.Module):
    def __init__(self, max_length):
        super().__init__()
        self.num_block = 3
        self.embed_dim = 512
        self.num_head  = 32
        self.max_length = max_length
        self.num_point = point_dim

        pos_embed = positional_encoding1(max_length, self.embed_dim)
        self.pos_embed = nn.Parameter(pos_embed)

        self.cls_embed = nn.Parameter(torch.zeros((1, self.embed_dim)))
        self.x_embed = XEmbed()

        self.norm = nn.LayerNorm(self.embed_dim)

        self.encoder = nn.ModuleList([
            TransformerBlock3(
                self.embed_dim,
                self.num_head,
                self.embed_dim
            ) for i in range(self.num_block)
        ])
        self.logit = nn.Linear(self.embed_dim, num_class)

    def forward(self, xyz):
        L,D = xyz.shape
        xyz = xyz.reshape(1,L,D)
        
        x = self.x_embed(xyz)
        
        x = x + self.pos_embed[:L].reshape(1,L,self.embed_dim)
        x = self.norm(x)
        x = torch.cat([
                self.cls_embed.reshape(1,1,self.embed_dim),
                x
            ],1)
        #x = x.unsqueeze(1)

        #for block in self.encoder: x = block(x) #remove tflite loop
        x = self.encoder[0](x)
        x = self.encoder[1](x)
        x = self.encoder[2](x)
        #x = F.dropout(x,p=0.4,training=False)

        logit = self.logit(x.mean(1))
        return logit

In [8]:
class FinalNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.m1 = SingleNet2(384)
        checkpoint_file1 = '/home/zhouyuanzhe/zhouplearn/asl/saved_models/checkpoint-best-score/val_all_3layerlessnoise9999smooth75bs128head16seef22222222moreaugchangedata_epoch_250.pth'

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        state_dict = torch.load(checkpoint_file1)['state_dict']
        for k, v in state_dict.items():
            new_state_dict[k.replace('module.','')] = v
        self.m1.load_state_dict(new_state_dict)


        self.m2 = SingleNet()
        checkpoint_file2 = '/home/zhouyuanzhe/zhouplearn/asl/saved_models/checkpoint-best-score/epoch_399.pth'

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        state_dict = torch.load(checkpoint_file2)['state_dict']
        for k, v in state_dict.items():
            new_state_dict[k.replace('module.','')] = v
        self.m2.load_state_dict(new_state_dict)


    def forward(self, xyz):
        logit = F.softmax(self.m1(xyz), -1)
        logit = logit + F.softmax(self.m2(xyz), -1)
        return logit

In [9]:
final_net = FinalNet()
final_net.eval()
1

1

In [10]:
import onnx
import onnx_tf
from onnx_tf.backend import prepare
import tensorflow as tf
import onnxsim


In [11]:
# !pip install tensorflow_probability

In [12]:
max_length = 130 #1601

In [13]:
class InputNet(tf.keras.layers.Layer):
    def __init__(self, ):
        super(InputNet, self).__init__()
        self.lip = tf.constant(SLIP)
        self.lhand = tf.constant(LHAND)
        self.rhand = tf.constant(RHAND)
        self.pose = tf.constant(SPOSE)
        self.reye = tf.constant(REYE)
        self.leye = tf.constant(LEYE)
        self.nose = tf.constant(NOSE)
        self.hand_start = tf.constant(HAND_START)
        self.hand_end = tf.constant(HAND_END)
        self.max_length = max_length

    def call(self, xyz):
        #x = xyz
        xyz = xyz[:,:,:2]
        L = len(xyz)
        if L > self.max_length:

            i = (L-self.max_length)//2
            xyz = xyz[i:i + self.max_length]  # center


        not_nan_xyz =  tf.reshape(xyz[~tf.math.is_nan(xyz)], (-1,2))
        xyz -= tf.math.reduce_mean (not_nan_xyz, axis=0, keepdims=True)   # noramlisation to common maen
        xyz /= tf.reduce_mean(tf.math.reduce_std (not_nan_xyz, axis=0, keepdims=True))
        
        x = tf.concat([
            tf.gather(xyz, self.lhand, axis=1),
            tf.gather(xyz, self.rhand, axis=1),
            tf.gather(xyz, self.lip, axis=1),
            tf.gather(xyz, self.pose, axis=1),
            tf.gather(xyz, self.reye, axis=1),
            tf.gather(xyz, self.leye, axis=1),
            tf.gather(xyz, self.nose, axis=1),
        ],1)
        return  x

In [14]:
class FeatNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.DIST_INDEX = DIST_INDEX
        self.LIP_DIST_INDEX = LIP_DIST_INDEX
        self.POSE_DIST_INDEX = POSE_DIST_INDEX
        self.EYE_DIST_INDEX = EYE_DIST_INDEX
  
    def forward(self, xyz):
        lhand = xyz[:, 0:21,:]#21
        rhand = xyz[:, 21:42,:]#21
        lip   = xyz[:, 42:62,:]#20
        pose = xyz[:, 62:70,:]#8
        reye = xyz[:, 70:78,:]#16
        leye = xyz[:, 78:86,:]#16
        nose = xyz[:, 86:90,:]#4


        x = torch.cat([xyz[1:,:len(LHAND+RHAND),:]-xyz[:-1,:len(LHAND+RHAND),:],torch.zeros((1,len(LHAND+RHAND),2))],0)
        
        ld = lhand[:,:,:2].reshape(-1,len(LHAND),1,2)-lhand[:,:,:2].reshape(-1,1,len(LHAND),2)
        ld = torch.sqrt((ld**2).sum(-1))
        ld = ld.reshape(-1,len(LHAND)*len(LHAND))[:,DIST_INDEX]
        
        rd = rhand[:,:,:2].reshape(-1,len(LHAND),1,2)-rhand[:,:,:2].reshape(-1,1,len(LHAND),2)
        rd = torch.sqrt((rd**2).sum(-1))
        rd = rd.reshape(-1,len(LHAND)*len(LHAND))[:,DIST_INDEX]
        
        lipd = lip[:,:,:2].reshape(-1,len(SLIP),1,2)-lip[:,:,:2].reshape(-1,1,len(SLIP),2)
        lipd = torch.sqrt((lipd**2).sum(-1))
        lipd = lipd.reshape(-1,len(SLIP)*len(SLIP))[:,LIP_DIST_INDEX]
        
        posed = pose[:,:,:2].reshape(-1,len(SPOSE),1,2)-pose[:,:,:2].reshape(-1,1,len(SPOSE),2)
        posed = torch.sqrt((posed**2).sum(-1))
        posed = posed.reshape(-1,len(SPOSE)*len(SPOSE))[:,POSE_DIST_INDEX]
        
        reyed = reye[:,:,:2].reshape(-1,len(REYE),1,2)-reye[:,:,:2].reshape(-1,1,len(REYE),2)
        reyed = torch.sqrt((reyed**2).sum(-1))
        reyed = reyed.reshape(-1,len(REYE)*len(REYE))[:,EYE_DIST_INDEX]
        
        leyed = leye[:,:,:2].reshape(-1,len(LEYE),1,2)-leye[:,:,:2].reshape(-1,1,len(LEYE),2)
        leyed = torch.sqrt((leyed**2).sum(-1))
        leyed = leyed.reshape(-1,len(LEYE)*len(LEYE))[:,EYE_DIST_INDEX]

        dist_hand=torch.sqrt(((lhand-rhand)**2).sum(-1))

        xyz = torch.cat([xyz.reshape(-1,(len(LHAND+RHAND+REYE+LEYE+NOSE+SLIP+SPOSE))*2), 
                            x.reshape(-1,(len(LHAND+RHAND))*2),
                            ld,
                            rd,
                            lipd,
                            posed,
                            reyed,
                            leyed,
                            dist_hand,
                            ],1)
        
        xyz[torch.isnan(xyz)] = 0
            
        return xyz

In [15]:
feat_net  = FeatNet()
feat_net.eval()

FeatNet()

In [16]:
#pytorch to onnx to tflite
fold_dir = './'
if 1:
    
    name='model' 
    input_onnx_file   = f'{fold_dir}/{name}.input.onnx'
    single_onnx_file  = f'{fold_dir}/{name}.single.onnx' 
    input_tf_file    = f'{fold_dir}/input_tf'
    single_tf_file   = f'{fold_dir}/single_tf'
    tf_file     = f'{fold_dir}/tf'
    tflite_file = f'{fold_dir}/{name}.tflite'

    def run_convert_onnx(): 
        if 1:
            torch.onnx.export(
                feat_net,
                #torch.jit.script(input_net),
                #torch.jit.trace(input_net, torch.zeros(100,num_landmark,3)),          # model being run 
                torch.zeros((100,90,2)), # model input (or a tuple for multiple inputs)
                input_onnx_file,             # where to save the model (can be a file or file-like object)
                export_params = True,        # store the trained parameter weights inside the model file
                opset_version = 12,          # the ONNX version to export the model to
                do_constant_folding=True,    # whether to execute constant folding for optimization 
                input_names =  ['inputs'],    # the model's input names
                output_names = ['outputs'],   # the model's output names
                dynamic_axes={
                    'inputs': {0: 'length'},
                    'output': {0: 'length'},
                },
                #verbose = True,
            )
            torch.onnx.export(
                final_net,         
                #torch.jit.script(single_net),
                #torch.jit.trace(single_net, torch.zeros(max_length,82,3)),           

                torch.zeros((max_length,point_dim)), 
                single_onnx_file,             
                export_params = True,         
                opset_version = 12, 
                do_constant_folding=True,      
                input_names =  ['inputs'],     
                output_names = ['outputs'],  
                dynamic_axes={
                    'inputs': {0: 'length'},
                },
                #verbose = True,
            )
            print('torch.onnx.export() passed !!')

        if 1:
            for f in [input_onnx_file, single_onnx_file]:
                print(1)
                if f is None: continue
                model = onnx.load(f)
                onnx.checker.check_model(model)
                #model_simple, check = onnxsim.simplify(model)
                onnx.save(model, f)
            print('onnx simplify() passed !!')


    def run_convert_tflite():
        if 1:
            tf_rep = prepare(onnx.load(input_onnx_file))
            tf_rep.export_graph(input_tf_file) 
            tf_rep = prepare(onnx.load(single_onnx_file))
            tf_rep.export_graph(single_tf_file) 
            print('tf_rep.export_graph() passed !!')

        if 1:
            class TFModel(tf.Module):
                def __init__(self):
                    super(TFModel, self).__init__()
                    #self.input  = tf.saved_model.load(input_tf_file)
                    self.input = InputNet()
                    self.feat = tf.saved_model.load(input_tf_file)
                    self.single = tf.saved_model.load(single_tf_file)
                    self.input.trainable = False
                    # self.single.trainable = False

                @tf.function(input_signature=[
                    tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
                ])
                def call(self, input):
                    y = {}
                    x = self.input(input)
                    x = self.feat(**{'inputs': x})['outputs']
                    y['outputs'] = self.single(**{'inputs': x})['outputs']
                    
                    return y

            tfmodel = TFModel()
            tf.saved_model.save(tfmodel, tf_file, signatures={'serving_default': tfmodel.call})
            print('tf.saved_model() passed !!')

        if 1:
            converter = tf.lite.TFLiteConverter.from_saved_model(tf_file)
            # converter.target_spec.supported_ops = [
            #     tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
            #     tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
            # ]
            #converter.optimizations = [tf.lite.Optimize.DEFAULT]
            #converter.target_spec.supported_types = [tf.FLOAT16]
            #converter.allow_custom_ops = True
            #converter.experimental_new_converter = True 
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_types = [tf.float16]
            tf_lite_model = converter.convert()
            with open(tflite_file, 'wb') as f:
                f.write(tf_lite_model)
            print('tflite convert() passed !!')
 
    run_convert_onnx()
    run_convert_tflite()

/home/zhouyuanzhe/anaconda3/envs/base2/lib/python3.8/site-packages/torch/onnx/utils.py:2029: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn(


============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

torch.onnx.export() passed !!
1
1
onnx simplify() passed !!
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-05-03 11:11:02.455286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:11:13.622900: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9216 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-05-03 11:11:13.624454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6767 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2023-05-03 11:11:13.625858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:loc

INFO:tensorflow:Assets written to: .//input_tf/assets


INFO:tensorflow:Assets written to: .//input_tf/assets


...

INFO:tensorflow:Assets written to: .//tf/assets


INFO:tensorflow:Assets written to: .//tf/assets


tf.saved_model() passed !!


2023-05-03 11:12:02.121073: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-03 11:12:02.121114: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-03 11:12:02.121813: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: .//tf
2023-05-03 11:12:02.153040: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-03 11:12:02.153078: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: .//tf
2023-05-03 11:12:02.232626: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-05-03 11:12:02.243335: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-03 11:12:02.439887: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: .//tf
2023-05-03 11:12:02.570253: I tensorflow

tflite convert() passed !!


In [17]:
!zip submission.zip  'model.tflite'
# !unzip -o submission.zip
# !ls

updating: model.tflite (deflated 8%)


In [19]:
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / 543)
    data = data.values.reshape(n_frames, 543, len(data_columns))
    return data.astype(np.float32)

def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic


train = pd.read_csv(f"./train.csv")
label_index = read_dict(f"./sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])

In [21]:
%%time
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter('model.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

preds = []
labels = []
count = 0
N = 2000 #train.shape[0]
exclude_path = []
for i in tqdm(range(N)):
    frames = load_relevant_data_subset(f'./{train.iloc[i].path}')
    output = prediction_fn(inputs=frames)

    sign = np.argmax(output["outputs"])
    topk = np.argsort(output['outputs'])[0][-1:]
    label = label_index[train.iloc[i].sign]
    if label in topk:
        count += 1
    else:
        exclude_path.append(train.iloc[i].path)

count/N


# 114

 87%|████████▋ | 1741/2000 [01:31<00:15, 16.79it/s]

In [ ]:
# np.load('exclude.npy')

In [ ]:
# 4 head
CPU times: user 34.8 s, sys: 1.78 s, total: 36.6 s
Wall time: 36.8 s

# 16head + view
CPU times: user 35.4 s, sys: 1.84 s, total: 37.2 s
Wall time: 37.6 s

# 16 head
CPU times: user 34.4 s, sys: 1.66 s, total: 36.1 s
Wall time: 36.2 s

SyntaxError: invalid syntax (1901384113.py, line 2)

In [ ]:
# print(output["outputs"])

In [ ]:
# %%time
# cnt=0
# for i in tqdm(range(400)):
#     frames = load_relevant_data_subset(f'/kaggle/input/asl-signs/{train.iloc[i].path}')
#     output = prediction_fn(inputs=frames)
#     sign = np.argmax(output["outputs"])
#     print(f"Predicted label: {index_label[sign]}, Actual Label: {train.iloc[i].sign}")
#     if index_label[sign]!=train.iloc[i].sign:
#         cnt+=1
# print(cnt)

In [ ]:
# [0.01085861 0.00581808 0.00685086 0.00304363 0.01479311 0.00817709
#  0.00340905 0.00318506 0.0233486  0.01050716 0.1883654  0.06354285
#  0.00429218 0.00845518 0.00856266 0.00674799 0.00605066 0.00663673
#  0.01101495 0.00473946 0.00718604 0.01581484 0.00979448 0.00933167
#  0.00909571 0.00761033 0.0079331  0.01132183 0.00546459 0.00705986
#  0.00446261 0.00805381 0.00862744 0.00863338 0.00812062 0.00676276
#  0.00807868 0.02296103 0.00857505 0.00721142 0.00628418 0.01281936
#  0.00548189 0.00735567 0.00574347 0.00762902 0.00878952 0.00886532
#  0.01273214 0.00704682 0.01344372 0.00995772 0.0073814  0.00659714
#  0.00734928 0.00888218 0.00814375 0.00859207 0.0080293  0.00954303
#  0.0176774  0.00564051 0.02074489 0.00685068 0.01218808 0.00627878
#  0.00417787 0.00859415 0.00817147 0.00301294 0.01126396 0.00508036
#  0.01058887 0.00715309 0.00879091 0.00931721 0.00505042 0.00747855
#  0.00890584 0.0099776  0.00793931 0.00610487 0.00978467 0.0113838
#  0.00602421 0.00689216 0.00653033 0.00553205 0.00650916 0.01165451
#  0.00894795 0.00780089 0.00688653 0.00819174 0.03024156 0.00330047
#  0.00574727 0.00616653 0.01027174 0.00659046 0.00588827 0.00415567
#  0.00988593 0.00802699 0.00456634 0.00853014 0.00969563 0.00664793
#  0.00753294 0.00557278 0.00765562 0.0061117  0.00961248 0.00826899
#  0.01247947 0.00554131 0.0138128  0.01092679 0.01032632 0.01741139
#  0.00958047 0.00666909 0.00370869 0.0104286  0.00786915 0.00149389
#  0.01268701 0.00562417 0.00593344 0.00859769 0.00469574 0.00894758
#  0.00675207 0.00438429 0.00722449 0.00883858 0.00632744 0.00602204
#  0.00777974 0.00490518 0.00606634 0.00746216 0.00522026 0.00602841
#  0.00656764 0.0111602  0.00662472 0.00631345 0.00751031 0.00889285
#  0.00786171 0.00789982 0.01060051 0.00874808 0.00476171 0.00731064
#  0.00933065 0.00540848 0.02670925 0.00531322 0.00992232 0.00717802
#  0.00599299 0.00933668 0.00482148 0.00589903 0.00574286 0.00494124
#  0.01233393 0.00907254 0.00595442 0.00576073 0.00596522 0.005933
#  0.00664493 0.00352007 0.00895852 0.00677059 0.01110536 0.00539469
#  0.00851405 0.01116983 0.00432136 0.00624733 0.00474382 0.00759205
#  0.01367113 0.00743047 0.00906641 0.00573219 0.00537325 0.01006344
#  0.00801374 0.00972718 0.00623355 0.00803888 0.00692571 0.00617468
#  0.00841703 0.00599053 0.00909205 0.01109888 0.01301423 0.00654494
#  0.01213645 0.00592066 0.00864201 0.00987625 0.00792791 0.00789566
#  0.0055953  0.00887042 0.0045177  0.00581646 0.00857393 0.00500618
#  0.00586388 0.00491259 0.0074577  0.00624605 0.0091049  0.10224412
#  0.00682174 0.0088692  0.00820932 0.00832468 0.00531415 0.0089981
#  0.00428656 0.00630823 0.02486216 0.00903836 0.00734655 0.06334254
#  0.00587949 0.00601918 0.00598095 0.00721327 0.00662056 0.02103801
#  0.0051178  0.01425657 0.00883084 0.00373367 0.0118778  0.5697484
#  0.01081768 0.00919196 0.00815717 0.00522287]